In [1]:
import requests
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import dotenv_values
import sqlite3
import sqlite_utils
import time
import statistics


In [2]:
mid_kr_puuid = "yke-_ZuAP9wM6s3tpe-FnVWLlnwI4qLuLRzz933HfpI_jzKcDa1YwXbWGpFC7WJek_rv4x_kXk2Y-A"

In [3]:
conn = sqlite3.connect('koren_soloq_db.db')
cursor = conn.cursor()
match_end_data = cursor.execute('''SELECT metadata, info FROM MATCH_DATA''').fetchall()
match_timeline_data = cursor.execute('''SELECT metadata, timelineinfo FROM MATCH_DATA''').fetchall()

conn.close()

In [4]:
#Filter only mid role games
list_match_end_data_whereMid = []
mid_matchs = []
for idx_match, match in enumerate(match_end_data) :
    match_info = json.loads(match[1])
    for idx, participant in enumerate(match_info['participants']):
        if (participant['puuid'] == mid_kr_puuid) and (participant['teamPosition'] == "MIDDLE"):
            list_match_end_data_whereMid.append(match)
            mid_matchs.append(idx_match)

list_match_timeline_data_whereMid = [match_timeline_data[i] for i in mid_matchs]

In [5]:
print(len(match_end_data))
print(len(list_match_end_data_whereMid))
print(len(list_match_timeline_data_whereMid))

600
505
505


In [6]:
#Info after game
kda_list = []
cs_min = []
for match in list_match_end_data_whereMid :


    match_metadata = json.loads(match[0])
    for idx, player_puuid in enumerate(match_metadata['participants']):
        if player_puuid == mid_kr_puuid:
            index_qing = idx

    match_info = json.loads(match[1])

    #KDA
    kills = match_info['participants'][index_qing]['kills']
    assists = match_info['participants'][index_qing]['assists']
    deaths = match_info['participants'][index_qing]['deaths']
    if deaths == 0 : deaths = 1
    kda_list.append((kills+assists)/deaths)

    #CS/MIN
    monster_cs = match_info['participants'][index_qing]['neutralMinionsKilled']
    minion_cs = match_info['participants'][index_qing]['totalMinionsKilled']
    cs_min.append((monster_cs + minion_cs) /  (match_info['gameDuration'] //60))




In [7]:
json.loads(list_match_end_data_whereMid[0][1])['participants'][0]['puuid']

'kKSPluUio3QQcAttyUdjul-d2m3rPrCRxxf4llXyzcwkwdtpnZ6PGHFY_NH7IQfgbgf-XPko4bZE8A'

In [31]:
json.loads(list_match_timeline_data_whereMid[0][1])

{'endOfGameResult': 'GameComplete',
 'frameInterval': 60000,
 'frames': [{'events': [{'realTimestamp': 1729145471528,
     'timestamp': 0,
     'type': 'PAUSE_END'}],
   'participantFrames': {'1': {'championStats': {'abilityHaste': 0,
      'abilityPower': 0,
      'armor': 35,
      'armorPen': 0,
      'armorPenPercent': 0,
      'attackDamage': 25,
      'attackSpeed': 100,
      'bonusArmorPenPercent': 0,
      'bonusMagicPenPercent': 0,
      'ccReduction': 0,
      'cooldownReduction': 0,
      'health': 670,
      'healthMax': 670,
      'healthRegen': 0,
      'lifesteal': 0,
      'magicPen': 0,
      'magicPenPercent': 0,
      'magicResist': 32,
      'movementSpeed': 340,
      'omnivamp': 0,
      'physicalVamp': 0,
      'power': 339,
      'powerMax': 339,
      'powerRegen': 0,
      'spellVamp': 0},
     'currentGold': 500,
     'damageStats': {'magicDamageDone': 0,
      'magicDamageDoneToChampions': 0,
      'magicDamageTaken': 0,
      'physicalDamageDone': 0,
     

In [37]:
json.loads(list_match_end_data_whereMid[0][1])['participants'][0]['teamId']

100

In [38]:
#Info timeline
gold_diff_15_list = []
for matchIndex, match in enumerate(list_match_timeline_data_whereMid):


    match_metadata = json.loads(match[0])
    for idx, player_puuid in enumerate(match_metadata['participants']):
        if player_puuid == mid_kr_puuid:
            index_qing = idx

    
    for idx, participant in enumerate(json.loads(list_match_end_data_whereMid[matchIndex][1])['participants']):
        if (participant['teamPosition'] == "MIDDLE") and (participant['puuid']) != mid_kr_puuid:
            index_ennemy = idx
            side_ennemy = participant['teamId']
            if side_ennemy == 100 : 
                side_qing = 200 
            else : 
                side_qing = 100
           

    match_info = json.loads(match[1])
    
    if len(match_info['frames']) > 16 :
        qing_gold_15 = match_info['frames'][15]['participantFrames'][str(index_qing+1)]['totalGold']
        ennemy_gold_15 = match_info['frames'][15]['participantFrames'][str(index_ennemy+1)]['totalGold']
        gold_diff_15_list.append(qing_gold_15 - ennemy_gold_15)

    for frame in match_info['frames'] :
        pass

# gold diff
# forward %

SyntaxError: invalid syntax (769299058.py, line 30)

In [81]:
#attention il a des games autofills
print(statistics.mean(kda_list))
print(statistics.mean(cs_min))
print(statistics.mean(gold_diff_15_list))

4.438730873087309
7.886367344876574
236.53493013972056
